In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

Let's start by getting familiar with our data.

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Task 1. Write a query that returns each genre, with the number of tracks sold in the USA:

* in absolute numbers
* in percentages.

In [3]:
%%sql

WITH genre_number AS

(SELECT 
    g.name genre_name,COUNT(g.name) genre_count
FROM customer 
INNER JOIN invoice ON invoice.customer_id=customer.customer_id
INNER JOIN invoice_line il ON il.invoice_id=invoice.invoice_id
INNER JOIN track t ON t.track_id=il.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
WHERE customer.country == "USA"
GROUP BY genre_name
ORDER BY genre_count DESC)

SELECT genre_name, genre_count, ROUND(CAST(genre_number.genre_count AS Float)*100/(SELECT CAST(SUM(genre_count) AS Float) FROM genre_number),2 ) genre_percentage

FROM genre_number


 * sqlite:///chinook.db
Done.


genre_name,genre_count,genre_percentage
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


We can clearly see that Rock is by far the the most popular genre sold in the USA, taking over 50% of total sales

| Artist Name | Genre |
|-------------|-------| 
|Regal | Hip-Hop |
|Red Tone |	Punk |
|Meteor and the Girls | Pop |
|Slim Jim Bites |	Blues |

Hence i would suggest that if we are told to choose three out of four albums above, I would omit  `Regal` from Hip-Hop section 

## Task 2. Find the total dollar amount of sales assigned to each sales support agent within the company. Add any extra attributes for that employee that you find are relevant to the analysis.

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.



Let's explore two relevant tables here: customer and employee:

In [4]:
%%sql
SELECT *
  FROM customer
LIMIT 10;   

 * sqlite:///chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
7,Astrid,Gruber,None,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,None,Austria,1010,+43 01 5134505,None,astrid.gruber@apple.at,5
8,Daan,Peeters,None,Grétrystraat 63,Brussels,None,Belgium,1000,+32 02 219 03 03,None,daan_peeters@apple.be,4
9,Kara,Nielsen,None,Sønder Boulevard 51,Copenhagen,None,Denmark,1720,+453 3331 9991,None,kara.nielsen@jubii.dk,4
10,Eduardo,Martins,Woodstock Discos,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,+55 (11) 3033-5446,+55 (11) 3033-4564,eduardo@woodstock.com.br,4


In [5]:
%%sql 
SELECT *
  FROM employee
WHERE title == 'Sales Support Agent'    
LIMIT 10;    

 * sqlite:///chinook.db
Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


There is a link between two tables via employee_id which is the same as support_rep_id in customer table

In [6]:
%%sql
SELECT *
  FROM customer
GROUP BY support_rep_id    

 * sqlite:///chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
59,Puja,Srivastava,None,"3,Raj Bhavan Road",Bangalore,None,India,560001,+91 080 22289999,None,puja_srivastava@yahoo.in,3
56,Diego,Gutiérrez,None,307 Macacha Güemes,Buenos Aires,None,Argentina,1106,+54 (0)11 4311 4333,None,diego.gutierrez@yahoo.ar,4
57,Luis,Rojas,None,"Calle Lira, 198",Santiago,None,Chile,None,+56 (0)2 635 4444,None,luisrojas@yahoo.cl,5


In [7]:
%%sql
SELECT  
      e.employee_id,
      e.first_name || ' ' ||e.last_name employee_name, 
      ROUND(SUM(invoice.total),2) total_sales_USD  
FROM customer c
INNER JOIN employee e ON e.employee_id=c.support_rep_id
INNER JOIN invoice ON invoice.customer_id = c.customer_id
GROUP BY support_rep_id
 

 * sqlite:///chinook.db
Done.


employee_id,employee_name,total_sales_USD
3,Jane Peacock,1731.51
4,Margaret Park,1584.0
5,Steve Johnson,1393.92


Jane Peacock is the strongest employee with total sales at $1731.51

## Task 3. Calculate data, for each country:

Write a query that collates data on purchases from different countries.
Where a country has only one customer, collect them into an "Other" group.
The results should be sorted by the total sales from highest to lowest, with the "Other" group at the very bottom.

For each country, include:
* total number of customers
* total value of sales
* average value of sales per customer
* average order value

In [8]:
%%sql

WITH 

customer_count AS

                    (SELECT COUNT(DISTINCT c.customer_id) no_of_customers,
                             c.country country,
                             ROUND(SUM(i.total),2) total_value_per_customer,
                             COUNT(i.total) no_of_sales_per_customer
                     FROM customer c
                     INNER JOIN invoice i ON i.customer_id=c.customer_id
                     GROUP BY 2),

country_grouping AS
                   (SELECT country, 
                           SUM(no_of_customers) no_of_customers, 
                           SUM(total_value_per_customer) total_sales,
                           ROUND(SUM(total_value_per_customer)/SUM(no_of_sales_per_customer),2)avg_order_value,
                           ROUND(SUM(total_value_per_customer)/SUM(no_of_customers),2) avg_per_customer,
       
                    CASE
                           WHEN no_of_customers == 1 THEN 'Other'
                           ELSE country
                    END AS country_cat

                    FROM customer_count
                    GROUP BY country_cat),

final_table AS
                  (SELECT country_cat country,
                          no_of_customers,
                          total_sales,
                          avg_order_value,
                          avg_per_customer,
                   CASE
                          WHEN country_cat = 'Other' THEN 1
                          ELSE 0
                   END AS sorting

                  FROM country_grouping
                  ORDER BY sorting ASC, total_sales DESC)

SELECT country,
       no_of_customers,
       total_sales,
       avg_order_value,
       avg_per_customer
        
FROM final_table        



 * sqlite:///chinook.db
Done.


country,no_of_customers,total_sales,avg_order_value,avg_per_customer
USA,13,1040.49,7.94,80.04
Canada,8,535.59,7.05,66.95
Brazil,5,427.68,7.01,85.54
France,5,389.07,7.78,77.81
Germany,4,334.62,8.16,83.66
Czech Republic,2,273.24,9.11,136.62
United Kingdom,3,245.52,8.77,81.84
Portugal,2,185.13,6.38,92.56
India,2,183.15,8.72,91.58
Other,15,1094.94,7.45,73.0


## Task 4. Write a query that categorizes each invoice as either an album purchase or not, and calculates the following summary statistics:
## - Number of invoices
## - Percentage of invoices

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

* purchase a whole album
* purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

In [9]:
%%sql

WITH 
invoice AS
                            (SELECT il.invoice_id invoice_id,
                            il.track_id track_id,
                            t.album_id album_id
                    FROM invoice_line il
                    INNER JOIN track t ON t.track_id=il.track_id 
                    GROUP BY 1),

sorting AS     
    (SELECT invoice.invoice_id, invoice.track_id, invoice.album_id,
CASE
WHEN
(SELECT il.track_id FROM invoice_line il
WHERE il.invoice_id=invoice.invoice_id
EXCEPT
SELECT t.track_id FROM track t
WHERE t.album_id=invoice.album_id) IS NULL
AND 
(SELECT t.track_id FROM track t
WHERE t.album_id=invoice.album_id
EXCEPT
SELECT il.track_id FROM invoice_line il
WHERE il.invoice_id=invoice.invoice_id)
IS NULL THEN 'yes'
ELSE 'no'
END AS SORTING

FROM invoice)
    
SELECT COUNT(DISTINCT invoice_id)no_of_invoices, 
       ROUND(((CAST(COUNT(DISTINCT invoice_id) AS Float))/(SELECT COUNT(invoice_id) FROM invoice))*100,2) percentage,
       SORTING album_purchase
FROM sorting
GROUP BY 3
 



 * sqlite:///chinook.db
Done.


no_of_invoices,percentage,album_purchase
500,81.43,no
114,18.57,yes


only 18.6% contributes to the total of album purchases.